## Preprocessing

In [15]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [16]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN', 'NAME'], inplace=True)

In [17]:
# Determine the number of unique values in each column.
print(application_df.nunique())


APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [20]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"

application_type_counts = application_df['APPLICATION_TYPE'].value_counts()

# Choose a cutoff value for determining rare types (e.g., 500)
cutoff = 500

# Identify application types that occur less than the cutoff
rare_application_types = application_type_counts[application_type_counts < cutoff].index.tolist()

# Replace rare application types with "Other"
application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(rare_application_types, 'Other')



In [34]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() <= 10].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [35]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
# Count the occurrences of each classification
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Choose a cutoff value for determining rare classifications (e.g., 100)
cutoff = 100

# Identify classifications that occur less than the cutoff
rare_classifications = classification_counts[classification_counts < cutoff].index.tolist()

# Replace rare classifications with "Other"
application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(rare_classifications, 'Other')

In [36]:
# You may find it helpful to look at CLASSIFICATION value counts >1
# Count the occurrences of each classification
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Filter for classifications that occur more than once
frequent_classifications = classification_counts[classification_counts > 1]

# Create a list of classifications to replace with "Other"
rare_classifications = classification_counts[classification_counts <= 1].index.tolist()

# Replace rare classifications with "Other"
application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(rare_classifications, 'Other')

In [39]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() <= 5].index.tolist()
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: count, dtype: int64

In [40]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_encoded = pd.get_dummies(application_df.drop('STATUS', axis=1))  # Drop target variable


In [41]:
# Split our preprocessed data into our features and target arrays
X = application_df_encoded.values
y = application_df['STATUS'].values  # Target variable

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [42]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [48]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer with ReLU activation (good for non-linear relationships)
nn.add(tf.keras.layers.Dense(units=32, activation='relu', input_shape=(X_train_scaled.shape[1],)))

# Second hidden layer with ReLU activation
nn.add(tf.keras.layers.Dense(units=16, activation='relu'))

# Output layer with sigmoid activation (suitable for binary classification)
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │         1,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,145 (8.38 KB)

 Trainable params: 2,145 (8.38 KB)

 Non-trainable params: 0 (0.00 B)

In [49]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])


In [50]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=100, verbose=1)


Epoch 1/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 479us/step - accuracy: 0.9028 - loss: 0.1852
Epoch 2/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 439us/step - accuracy: 0.9998 - loss: 0.0022
Epoch 3/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 446us/step - accuracy: 0.9998 - loss: 0.0018
Epoch 4/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 450us/step - accuracy: 0.9999 - loss: 8.6202e-04
Epoch 5/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 482us/step - accuracy: 0.9996 - loss: 0.0022
Epoch 6/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 453us/step - accuracy: 0.9999 - loss: 8.3027e-04
Epoch 7/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 453us/step - accuracy: 0.9998 - loss: 0.0013
Epoch 8/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 449us/step - accuracy: 0.9996 - loss: 0.0021
Epoch 9/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 453us/step - accuracy: 0.9997 - loss: 0.0019
Epoch 10/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 490us/step - accuracy: 0.9997 - loss: 0.0013
Epoch 11/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 455us/step - accuracy: 0.9997 - loss: 0.0018
Epoch 12/10

In [51]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 682us/step - accuracy: 1.0000 - loss: 1.9389e-04
Loss: 0.00019389415683690459, Accuracy: 1.0


In [53]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity.h5')
